In [3]:
import pandas as pd
import json

# 이벤트 기간에 가입한 유저

### 전체 가입 유저

In [ ]:
# reward_test_all_user.csv 산출 쿼리
select date_format(u.join_date, '%Y-%m-%d') as join_date, u.id as uid
from user u
where u.join_date between '2019-10-21 00:00:00' and '2019-10-23 23:59:59'

In [9]:
read_path = '../csv/reward_test_all_user.csv'
all_user = pd.read_csv(read_path)

all_user['uid'] = all_user['uid'].astype(str)

all_user.tail()

,join_date,uid
19949,2019-10-23,10117330
19950,2019-10-23,10117331
19951,2019-10-23,10117332
19952,2019-10-23,10117333
19953,2019-10-23,10117334


### 이벤트 대상 유저

In [8]:
read_path = '../csv/reward_test_target_user.csv'
target_user = pd.read_csv(read_path)

target_user = target_user[target_user['전송 실패'] != '전송 실패'] # 번개톡 전송 실패 제외

target_user = target_user.rename(columns={'date': 'event_date'})

target_user = target_user[['event_date', 'uid', 'test_group']]

target_user['uid'] = target_user['uid'].astype(str)

target_user

,event_date,uid,test_group
0,2019-10-22,10096135,group_3
1,2019-10-22,10096136,group_0
2,2019-10-22,10096138,group_2
3,2019-10-22,10096139,group_3
4,2019-10-22,10096140,group_0
...,...,...,...
5337,2019-10-24,10117321,group_1
5338,2019-10-24,10117322,group_2
5339,2019-10-24,10117329,group_1
5340,2019-10-24,10117330,group_2


### 전체 가입 유저와 이벤트 대상 유저 merge

In [10]:
user = pd.merge(all_user, target_user, on='uid', how='left')

def group_reward(row):
    if row['test_group'] == 'group_0':
        return '2000'
    elif row['test_group'] == 'group_1':
        return '4000'
    elif row['test_group'] == 'group_2':
        return '6000'
    elif row['test_group'] == 'group_3':
        return '8000'
    else:
        return '0'

user['reward'] = user.apply(group_reward, axis=1)
    
user

,join_date,uid,event_date,test_group,reward
0,2019-10-21,10096134,NaN,NaN,0
1,2019-10-21,10096135,2019-10-22,group_3,8000
2,2019-10-21,10096136,2019-10-22,group_0,2000
3,2019-10-21,10096137,NaN,NaN,0
4,2019-10-21,10096138,2019-10-22,group_2,6000
...,...,...,...,...,...
19949,2019-10-23,10117330,2019-10-24,group_2,6000
19950,2019-10-23,10117331,NaN,NaN,0
19951,2019-10-23,10117332,2019-10-24,group_0,2000
19952,2019-10-23,10117333,NaN,NaN,0


In [11]:
user.groupby(['test_group'], as_index=False).agg({'uid': 'count'})

,test_group,uid
0,group_0,1323
1,group_1,1326
2,group_2,1308
3,group_3,1374


# 이벤트 기간에 등록된 상품

In [1]:
# reward_test_product.csv 산출 쿼리
select u.uid, p.create_date, p.id as pid
from (
    select distinct u.id as uid
    from user u
    where u.status = 0 # 탈퇴 유저 제외
        and u.join_date between '2019-10-21 00:00:00' and '2019-10-23 23:59:59' # 이벤트 대상 가입일
) u
join product_info p
on u.uid = p.uid
where p.create_date between '2019-10-22 00:00:00' and '2019-10-31 23:59:59' # 상품 등록 기간

SyntaxError: invalid syntax (<ipython-input-1-9a87f8fc92c0>, line 2)

In [56]:
read_path = '../csv/reward_test_product.csv'
product_raw = pd.read_csv(read_path)

product_raw['uid'] = product_raw['uid'].astype(str)
product_raw['pid_str'] = product_raw['pid'].astype(str)

product_raw

,uid,create_date,pid,pid_str
0,10096140,2019-10-26 19:24:00.854525,110631153,110631153
1,10096192,2019-10-22 01:10:52.643629,110365794,110365794
2,10096192,2019-10-22 11:20:37.584432,110376493,110376493
3,10096218,2019-10-25 18:56:43.725892,110577984,110577984
4,10096227,2019-10-23 18:10:04.888749,110460098,110460098
...,...,...,...,...
5314,10117269,2019-10-26 09:41:38.462283,110604546,110604546
5315,10117269,2019-10-24 00:05:36.885096,110482721,110482721
5316,10117269,2019-10-24 00:04:55.972227,110482683,110482683
5317,10117282,2019-10-23 23:58:46.609911,110482395,110482395


In [86]:
#유저별 등록한 상품 수
product_uid = product_raw.groupby(['uid'], as_index=False).agg({'pid': 'count'})

product_uid = product_uid.rename(columns={'pid': 'products'})

product_uid

,uid,products
0,10096140,1
1,10096192,2
2,10096218,1
3,10096227,2
4,10096228,2
...,...,...
1496,10117243,1
1497,10117260,1
1498,10117269,3
1499,10117282,1


### merge 테이블 만들기

In [92]:
user_product = pd.merge(user, product_uid, on='uid', how='left')

user_product

,join_date,uid,event_date,test_group,reward,products
0,2019-10-21,10096134,NaN,NaN,0,NaN
1,2019-10-21,10096135,2019-10-22,group_3,8000,NaN
2,2019-10-21,10096136,2019-10-22,group_0,2000,NaN
3,2019-10-21,10096137,NaN,NaN,0,NaN
4,2019-10-21,10096138,2019-10-22,group_2,6000,NaN
...,...,...,...,...,...,...
19949,2019-10-23,10117330,2019-10-24,group_2,6000,NaN
19950,2019-10-23,10117331,NaN,NaN,0,NaN
19951,2019-10-23,10117332,2019-10-24,group_0,2000,NaN
19952,2019-10-23,10117333,NaN,NaN,0,NaN


In [88]:
user_product['products'].max()

105.0

### 유저 그룹별 상품 등록 수

In [93]:
product_group = user_product_uid.groupby(['join_date', 
                                          'reward'], as_index=False).agg({'uid':'count', 
                                                                          'products':['count', 'sum']})

product_group

join_date reward   uid products        
                      count    count     sum
0   2019-10-21      0  5059      195   767.0
1   2019-10-21   2000   453       38   107.0
2   2019-10-21   4000   476       36   250.0
3   2019-10-21   6000   444       32   124.0
4   2019-10-21   8000   514       34   172.0
5   2019-10-22      0  4739      315  1057.0
6   2019-10-22   2000   413       48   235.0
7   2019-10-22   4000   413       72   339.0
8   2019-10-22   6000   437       58   250.0
9   2019-10-22   8000   454       71   171.0
10  2019-10-23      0  4825      341   996.0
11  2019-10-23   2000   457       58   158.0
12  2019-10-23   4000   437       82   293.0
13  2019-10-23   6000   427       56   206.0
14  2019-10-23   8000   406       65   194.0

# 이벤트 기간의 채팅

In [20]:
read_path = '../csv/reward_test_chat.csv'
chat_raw = pd.read_csv(read_path)

chat_raw.tail()

,extras,channel_id
1179219,"{""pid"":105058524,""thumbnailUrl"":""https:\/\/seo...",154892771
1179220,"{\n ""uid"" : ""3196841"",\n ""price"" : ""250,000 ...",167407487
1179221,"{""pid"":97692564,""thumbnailUrl"":""https:\/\/seou...",167410570
1179222,"{\n ""ref"" : ""검색결과"",\n ""uid"" : ""149715"",\n ""...",167410572
1179223,"{\n ""ref"" : ""검색결과"",\n ""uid"" : ""10168863"",\n ...",167410581


In [97]:
def extras_to_pid(row):
    return json.loads(row['extras']).get('pid')
        
chat_raw['pid'] = chat_raw.apply(extras_to_pid, axis=1)

# pid별 채팅 수 산출
product_chat = chat_raw.groupby(['pid'], as_index=False).agg({'channel_id': pd.Series.nunique})

product_chat['pid'] = product_chat['pid'].astype(str)

product_chat = product_chat.rename(columns={'channel_id': 'chats'})

product_chat.tail()

KeyboardInterrupt: 

In [95]:
chat['channel_id'].max()

486

In [94]:
user_product_chat = pd.merge(product_raw, product_chat, left_on='pid_str', right_on='pid', how='left')

user_product_chat.drop(['pid_str', 'pid_y'], axis=1, inplace=True)

user_product_chat = user_product_chat.rename(columns={'pid_x': 'pid'})

user_product_chat['pid'] = user_product_chat['pid'].astype(str)

user_product_chat

,uid,create_date,pid,channel_id
0,10096140,2019-10-26 19:24:00.854525,110631153,NaN
1,10096192,2019-10-22 01:10:52.643629,110365794,2.0
2,10096192,2019-10-22 11:20:37.584432,110376493,NaN
3,10096218,2019-10-25 18:56:43.725892,110577984,1.0
4,10096227,2019-10-23 18:10:04.888749,110460098,1.0
...,...,...,...,...
5896,10117269,2019-10-24 00:05:36.885096,110482721,1.0
5897,10117269,2019-10-24 00:05:36.885096,110482721,1.0
5898,10117269,2019-10-24 00:04:55.972227,110482683,NaN
5899,10117282,2019-10-23 23:58:46.609911,110482395,NaN


In [ ]:
pid_chat = user_product_chat.groupby(['pid'])

In [81]:
type(user_product_chat['pid'][4])

numpy.int64

# 이벤트 기간의 번프

In [74]:
read_path = '../csv/reward_test_bunp.csv'
bunp_raw = pd.read_csv(read_path)

bunp_raw['seller_pid'] = bunp_raw['seller_pid'].fillna(0)

bunp_raw['seller_pid'] = bunp_raw['seller_pid'].astype(int).astype(str)

bunp_raw.tail()

,channel_id,seller_uid,buyer_uid,seller_pid,created_at
171631,167396305,4331758,7761492,105399623,2019-10-31 23:59:29
171632,167346340,8183196,10160952,109173387,2019-10-31 23:59:37
171633,165219214,1358510,7242952,80910784,2019-10-31 23:59:39
171634,167410296,3566971,3773382,105578884,2019-10-31 23:59:45
171635,167264000,4874182,4831491,110632324,2019-10-31 23:59:49


In [84]:
bunp_pid = bunp_raw.groupby(['seller_pid'], as_index=False).agg({'channel_id': pd.Series.nunique})

bunp_pid = bunp_pid.rename(columns={'channel_id': 'bunp'})

bunp_pid

,seller_pid,bunp
0,0,30
1,100000180,1
2,100001266,1
3,100001311,1
4,100002703,1
...,...,...
145211,99997293,1
145212,99997333,1
145213,99998647,1
145214,99998714,1


In [85]:
user_product_chat_bunp = pd.merge(user_product_chat, bunp_pid, left_on='pid', right_on='seller_pid', how='left')

user_product_chat_bunp

,uid,create_date,pid,channel_id,seller_pid,bunp
0,10096140,2019-10-26 19:24:00.854525,110631153,NaN,NaN,NaN
1,10096192,2019-10-22 01:10:52.643629,110365794,2.0,110365794,1.0
2,10096192,2019-10-22 11:20:37.584432,110376493,NaN,NaN,NaN
3,10096218,2019-10-25 18:56:43.725892,110577984,1.0,NaN,NaN
4,10096227,2019-10-23 18:10:04.888749,110460098,1.0,110460098,1.0
...,...,...,...,...,...,...
5896,10117269,2019-10-24 00:05:36.885096,110482721,1.0,NaN,NaN
5897,10117269,2019-10-24 00:05:36.885096,110482721,1.0,NaN,NaN
5898,10117269,2019-10-24 00:04:55.972227,110482683,NaN,NaN,NaN
5899,10117282,2019-10-23 23:58:46.609911,110482395,NaN,NaN,NaN


In [ ]:
event_funnel = user_product_chat_bunp.groupby(['join_date', 'reward'], as_index=False).agg({'uid':'count', 
                                                                          'pid':['count', 'sum']})
